In [49]:
import numpy as np
import pandas as pd
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import linear_kernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf
from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
from sentence_transformers import SentenceTransformer


In [50]:
# RNN은 데이터 --> Embedding Layer(특성, 단어 임베딩) --> RNNLayer(문장임베딩) --> Dense layer

# 임베딩레이어와 RNNLayer는 특성 추출임

# 추출한것을 문장당 몇개의 숫자를 넘겨주면 그 숫자값을 입력으로 해서 Dense layer에서 분류

# 앞단계는 특성추출, 뒷단계는 분류

# CNN도 비슷하다!

## CNN

In [51]:
# CNN(Convolutional Neural Network) - 구성요소
'''

특징 추출

1. Convolution Layer (합성곱 계층) : 합성곱 연산
2. Pooling Layer(풀링 계층) : 풀링 연산
줄여서 Feature Map

분류

Fully Connected Layer (완전 연결 계층) : 물체 분류

'''

# 다중분류면 softmax 함수, 단일분류(이중)이면 sigmoid 함수

'\n\n특징 추출\n\n1. Convolution Layer (합성곱 계층) : 합성곱 연산\n2. Pooling Layer(풀링 계층) : 풀링 연산\n줄여서 Feature Map\n\n분류\n\nFully Connected Layer (완전 연결 계층) : 물체 분류\n\n'

In [52]:
# 1-1. 합성곱 계층(Convolution Layer) < 핵심
# 역할 : 합성곱 연산을 통해 이미지에서 특징을 추출
# 특징(feature) : 물체들을 구별할 수 있는 성질 예) 윤곽선, 귀의 모양, 얼굴의 형태
# 필터(filter, mask, kernel) : 3X3 또는 5x5크기의 행렬(2차원 배열)
# - 원본이미지를 변환한다의 의미, 필터
# - 가린다는 마스크

In [53]:
# 합성곱 연산 후 이미지의 크기

# 입력 이미지의 크기 (H, W)
# 필터의 크기 (FH, FW)
# 특성맵의 크기 (OH, OW)
# -->

# OH = ( H - FH ) +1
# OW = ( W - FW) +1

'''

strides가 2일때와 비교하면 1일때보다 결과값이
strides가 2일 때 정확하게 2배로 줄어든다

'''

'\n\nstrides가 2일때와 비교하면 1일때보다 결과값이\nstrides가 2일 때 정확하게 2배로 줄어든다\n\n'

In [54]:
# 컬러라서 3채널.RGB

# 입력 채널이 3채널이면 필터도 3채널이어야 함

# 입력 채널 수와 필터의 채널 수가 같아야 함

# (4, 4, 3) (3, 3, 3) = (1+1) (1+1) Feature맵은 2차원으로 나옴. 

In [55]:
# 3차원 데이터의 합성곱 : 블록(cube)으로 생각하기
# 입력 데이터와 필터를 아래의 그림처럼 직육면체의 블록으로 생각하면 쉬움.
# 3차원 데이터의 모양은 (높이, 너비, 채널) = (Height, Width, Channel) 순으로 표현함

# 필터수만큼 피쳐맵이 나옴

In [56]:
# OpenCV와 딥러닝 중 어떤 필터가 최선일까?

# 1. OpenCV : 가중치(필터의 숫자 값)을 사람이 결정함.
# 2. 딥러닝 : 랜덤한 숫자 값에서 출발
# -> 학습을 통해 최적의 가중치를 찾음
# -> 딥러닝에서 학습을 한다는 것은 필터의 가중치 값을 찾는다는 것

## 패딩(Padding)

In [57]:
# 패딩(padding)
# - 개념 : 합성곱 연산을 수행하기 전에 입력데이터 주변을 특정 값(주로 0)으로 채워넣는 것.
# - 목적 : feature map(특성 맵)의 크기를 입력 데이터의 크기와 동일하게 만드는 것.

# (4,4) padding (6,6)
# (3,3)을 기준으로 convolution 연산의 결과 (4,4)이 됨.

# 앞에서 봤던 특성들이 아닌 다른 특성들을 더 뽑을 수 있을때까지 돌려보는 것.

## 풀링(Pooling)

In [58]:
'''

1. 풀링(Pooling)
개념 : feature map(특성 맵)의 가로, 세로 방향의 크기를 줄이는 연산
종류 : Max Pooling, Average Pooling
효과 : 데이터 압축, 불필요한 노이즈 제거

'''

# Windows Size와 Stride 똑같이 연동돼서 간다!

# (4,4) -> (2,2) # 픽셀값이 4분의1로 줄어든다. 4x4=16, 2x2=4...

# 불필요한 노이즈가 제거된다, 데이터 압축

# 입력 신호의 값을 그대로 출력하는 것이 아님

'\n\n1. 풀링(Pooling)\n개념 : feature map(특성 맵)의 가로, 세로 방향의 크기를 줄이는 연산\n종류 : Max Pooling, Average Pooling\n효과 : 데이터 압축, 불필요한 노이즈 제거\n\n'

In [59]:
# 마지막 처리, 출력층의 활성화 함수 - 이진분류

'''

목적에 따라 이진분류, 다중분류, 활성화함수를 다르게 선택하면 되는 것이고,

딱 0일 때 1/2이다.

0이상이면 >_1/2 그냥 1로 수렴시킴

0이하이면 <-1/2 그냥 0으로 수렴시킴

'''

# 이미지분류나 텍스트분류나 특성추출 RNN CNN은 다르지만

# 그 결과값은 넘어와서 Dense Layer에서 처리되기는 마찬가지다!

'\n\n목적에 따라 이진분류, 다중분류, 활성화함수를 다르게 선택하면 되는 것이고,\n\n딱 0일 때 1/2이다.\n\n0이상이면 >_1/2 그냥 1로 수렴시킴\n\n0이하이면 <-1/2 그냥 0으로 수렴시킴\n\n'

In [60]:
# 출력층의 활성화 함수 - 다중분류

'''
1.3                   0.02
5.1                   0.90
2.2                   0.05
0.7                   0.01
1.1                   0.02
우측값의 합이 1.9
'''

'\n1.3                   0.02\n5.1                   0.90\n2.2                   0.05\n0.7                   0.01\n1.1                   0.02\n우측값의 합이 1.9\n'

In [61]:
# 은닉층의 활성화 함수
'''
ReLU 함수 : 입력 값이 음수면 0으로 비활성화하고, 입력 값이 양수면 해당 값을 그대로 출력
            y=x 입력이 그대로 출력, 입력의 총합이 0이상일 때,
            
            tanh(하이퍼블릭 탄젠트) 함수 : 시그모이드 함수와 비슷, 출력 값이 -1과 1사이에 존재
            
'''

'\nReLU 함수 : 입력 값이 음수면 0으로 비활성화하고, 입력 값이 양수면 해당 값을 그대로 출력\n            y=x 입력이 그대로 출력, 입력의 총합이 0이상일 때,\n            \n            tanh(하이퍼블릭 탄젠트) 함수 : 시그모이드 함수와 비슷, 출력 값이 -1과 1사이에 존재\n            \n'

## tensorflow.keras로 CNN 구현하기

In [62]:
# RGB input image --> tf.keras.layers.Conv2D() --> tf.keras.layers.MaxPool2D()
#                                                 tf.keras.layers.AveragePooling2D()
#                                                 tf.keras.layers.Dense()
                                                
'''

Conv2D에 ReLU 함수 사용, tanh 사용

Dense에서는 이진이면 Sigmoid, 다중이면 SoftMax사용

'''                                                

'\n\nConv2D에 ReLU 함수 사용, tanh 사용\n\nDense에서는 이진이면 Sigmoid, 다중이면 SoftMax사용\n\n'

## CNN 모델을 이용한 이미지 분류 실습

In [63]:
# tf.keras에서 제공하는 MNIST dataset을 이용하여 손글씨 이미지를 분류하는 실습

# 1) 손으로 쓰여진 0부터 9까지의 숫자 이미지들로 구성
# 2) 흑백 이미지, 모양 = (28, 28)
# 3) 전체 데이터 셋 = train_data 60,000개 + test_data 10,000개

In [64]:
'''

CNN 모델 구성

입력 이미지 - Conv2D(ReLU) - MaxPooling 2D(ReLU) - Conv2D(은닉충-ReLU) - 

MaxPooling2D(Softmax) - Dense - 출력(예측)

'''
import tensorflow as tf

# add() 함수를 이용하여 레이어 추가 -> model.add(매개변수 = Layer)
# model.add(tf.keras.layers.Conv2D())
# model.add(tf.keras.layers.MaxPool2D())
# model.add(tf.keras.layers.flatten())
# model.add(tf.keras.layers.Dense())

In [65]:
# 함수 사용법 : Conv2D() 함수
# 용도 : 컨볼루션 계층을 생성하는 함수

'''
tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=1,
                       padding='same', activation='ReLU', input_shape(28,28,1))
'''

# 2D의 의미는 필터가 두 방향으로 움직이니까 그 방향성 때문에 2D임.
# Conv1D는 한방향으로 특성을 추출한다! <얘는 임베딩 벡터값에 대해서
# 컨볼루션 특성을 추출, 
# Conv2D, MaxPool2D, Conv2D, Maxpool2D 두번 반복해서 쓸지 한번만해서 쓸지는 본인 재량                     

"\ntf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=1,\n                       padding='same', activation='ReLU', input_shape(28,28,1))\n"

In [66]:
# Flatten() 함수
# Feature맵을 1차원으로 변환하는 함수

# tf.keras.layers.Flatten() --> LSTM의 마지막 형태랑 비슷함

'''

Dense(units = 10, activation='softmax') /
Dense(units = 1, activation='sigmoid'

'''

# 출력유닛이 하나일때만 Sigmoid, 2개이상일 경우엔 Softmax로 DenseLayer에서 사용

"\n\nDense(units = 10, activation='softmax') /\nDense(units = 1, activation='sigmoid'\n\n"

## Data Pre-Processing

In [67]:
# MNIST dataset 생성하기

tf.keras.datasets.mnist.load_data(path='mnist.npz')

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# 실습 전 마지막 전처리
'''

이미지의 픽셀 값을 0과 1 사이로 변경
1) 이미지 : 0~255 사이의 숫자로 존재
2) 문제점 : 숫자 크기의 차이로 인해 학습 시 왜곡이 발생 ( Scaling = Normalizing)

'''

# 스케일링(노말라이징)이 필요함

''' 0과 255사이의 숫자를 0과 1 사이의 숫자로 변경시키는 행위 '''

11490434/11490434 [==============================] - 1s 0us/step


' 0과 255사이의 숫자를 0과 1 사이의 숫자로 변경시키는 행위 '

In [69]:
from tensorflow import keras

# 커널의 크기는 파라미터(매개변수)임. 따라서 여러 가지 값을 시도해 봐야 함.
# 보통 (3,3)이나 (5,5)를 많이 씀

tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3), activation='ReLU',
                       padding='same', strides=1)

## 패션 MNIST 데이터 불러오기

In [ ]:
# 완전 연결 신경망에서는 입력 이미지를 밀집층에 연결하기 위해 일렬로 펼쳐야함.
# 이 작업을 위해 np.reshape() 메서드를 사용하거나 Flatten 클래스를 사용함.
# 합성곱 신경망은 2차원 이미지를 그대로 사용하기 때문에 일렬로 펼치지 않음.
''' 일렬로 펼치는 경우? 질문사항 '''
# 입력 이미지는 항상 깊이(채널)가 있어야 함
# 흑백 이미지의 경우 채널 차원이 없는 2차원 배열이지만
# COnv2D층을 사용하기 위해 마지막에 이 채널 자원을
# 추가해야함. np.reshape() 메서드를 사용해 전체 배열 차원을 그대로 유지하면서
# 마지막에 차원을 간단하게 추가할 수 있음.

In [71]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()
    
train_scaled = train_input.reshape(-1, 28, 28, 1) / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

''' (48000, 28, 28, 1) 크기를 어떻게 알 수 있나..? (-1, 28, 28, 1)의 의미..



4422102/4422102 [==============================] - 2s 0us/step
